In [1]:
import pandas as pd
import numpy as np

import os
from urllib.request import urlopen, Request
import requests as r
from sys import argv, exit
from base64 import b64encode
import json
import re

from dotenv import load_dotenv, find_dotenv
dotenv_path=find_dotenv()
load_dotenv(dotenv_path)
client_id= os.environ.get('SPOTIFY_CLIENT_ID')
client_secret= os.environ.get('SPOTIFY_CLIENT_SECRET')

import spotipy
import sys
from sys import argv, exit
from spotipy.oauth2 import SpotifyClientCredentials

#Authentication without user
client_credentials_manager= SpotifyClientCredentials(client_id= client_id, client_secret=client_secret)
sp= spotipy.Spotify(client_credentials_manager=client_credentials_manager)

We are going to be building models based 

In [2]:
df_collab= pd.read_csv('/Users/josephlim/Desktop/Data Science/Capstone Projects/Music Recommendation System- Capstone 3/Data/Raw data/spotify_dataset.csv', on_bad_lines='skip')
df_content=pd.read_csv('/Users/josephlim/Desktop/Data Science/Capstone Projects/Music Recommendation System- Capstone 3/Data/Raw data/songs_normalize.csv')

In [3]:
df_collab.shape

(12891680, 4)

This is a sizeable data set! Let's get it!

In [4]:
df_collab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12891680 entries, 0 to 12891679
Data columns (total 4 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          object
 1    "artistname"    object
 2    "trackname"     object
 3    "playlistname"  object
dtypes: object(4)
memory usage: 393.4+ MB


It makes sense that our features are all object type data.

In [5]:
df_collab.columns

Index(['user_id', ' "artistname"', ' "trackname"', ' "playlistname"'], dtype='object')

We have quotes and white spaces as part of our column names. Let's clean them up.

In [6]:
with_quote= df_collab.columns.to_list()
columns =[x.replace('"','') for x in with_quote]
columns_nowhite= [x.strip() for x in columns]

col_replace= dict(zip(with_quote, columns_nowhite))
df_collab= df_collab.rename(columns=col_replace)

In [7]:
df_collab.columns

Index(['user_id', 'artistname', 'trackname', 'playlistname'], dtype='object')

Let's also make sure our data doesn't have white spaces.

In [8]:
for i in columns_nowhite:
    df_collab[i].str.strip()

Now, that's what I like to see! Let's continue.

In [9]:
df_collab.head()

,user_id,artistname,trackname,playlistname
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010


In [10]:
df_collab.isna().any()

user_id         False
artistname       True
trackname        True
playlistname     True
dtype: bool

We have a bit of missing data value. Let's explore a bit and decide what to do with them.

In [11]:
df_collab.isna().sum()

user_id             0
artistname      33568
trackname          85
playlistname     1246
dtype: int64

There isn't any way to replace playlist information for those tracks without playlist names. Let's drop them.

In [12]:
df_collab_w_playlist= df_collab[df_collab.playlistname.notna()]

In [13]:
df_collab_w_playlist.isna().sum()

user_id             0
artistname      33568
trackname          85
playlistname        0
dtype: int64

In [14]:
df_notrack= df_collab_w_playlist[df_collab_w_playlist.trackname.isna()]

In [15]:
df_notrack.head(50)

,user_id,artistname,trackname,playlistname
224037,42f5289bfa83726edd652392ea09984a,NaN,NaN,Starred
268384,48388a944d86ca079dac6e5d825a2b57,Silversun Pickups,NaN,No One Sleeps When I’m Awake
276878,317a0f3ff15ff0cd8b12fe06f390a24f,Toufic Farroukh,NaN,Lounge
459375,db0d3d755f35fa0ed3985a0be1df0e49,Tamia,NaN,2
459754,db0d3d755f35fa0ed3985a0be1df0e49,Krezip,NaN,moi
459927,db0d3d755f35fa0ed3985a0be1df0e49,Tamia,NaN,relax
569216,c0cf65e23e3df6f75d60f26af75c7162,NaN,NaN,Beat Rush
570028,c0cf65e23e3df6f75d60f26af75c7162,NaN,NaN,Groovin'
846965,798ddeb5ce830765d64b1ff2de51660d,蘇永康,NaN,7- Shanghai
1181985,a21cb7091c1bd79f7b21414b1d07ebdf,NaN,NaN,Fuckin' electro-pop-rockin' stuff


We need both track names and artist names to be able to recommend songs, because there are many songs with same titles, and even the same artists don't always make the same types of music. Let's drop data that miss them.

In [16]:
df_collab_track= df_collab_w_playlist[df_collab_w_playlist.trackname.notna()] 
df_collab_name=df_collab_track[df_collab_track.artistname.notna()]

In [17]:
df_collab.isna().sum()

user_id             0
artistname      33568
trackname          85
playlistname     1246
dtype: int64

The data we'll use to build collaborative filtering model is clean. Let's rename some of our columns to match those in dataset to be used for content-based filtering model.

In [18]:
rename_dict= {'artistname':'artist','trackname':'song'}
df_collab_name.rename(columns=rename_dict, inplace=True)

/var/folders/z9/t15ddpld70nc5qf41442lpr80000gn/T/ipykernel_3389/2004673095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_collab_name.rename(columns=rename_dict, inplace=True)


In [19]:
df_collab_name.columns

Index(['user_id', 'artist', 'song', 'playlistname'], dtype='object')

In [20]:
df_collab_final=df_collab_name

Let's move onto data we'll use for content-based filtering model. 

### Dataset for content-based recommendation system

In [21]:
df_content.shape

(2000, 18)

In [22]:
df_content.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


We see that some data in release_date feature is in yyyy-mm-dd format, while others are in years. Let's put all of them into years.

In [23]:
df_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            2000 non-null   object 
 1   song              2000 non-null   object 
 2   duration_ms       2000 non-null   int64  
 3   explicit          2000 non-null   bool   
 4   year              2000 non-null   int64  
 5   popularity        2000 non-null   int64  
 6   danceability      2000 non-null   float64
 7   energy            2000 non-null   float64
 8   key               2000 non-null   int64  
 9   loudness          2000 non-null   float64
 10  mode              2000 non-null   int64  
 11  speechiness       2000 non-null   float64
 12  acousticness      2000 non-null   float64
 13  instrumentalness  2000 non-null   float64
 14  liveness          2000 non-null   float64
 15  valence           2000 non-null   float64
 16  tempo             2000 non-null   float64


All the datatypes look good. Let's check for missing values.

In [24]:
df_content.isna().any()

artist              False
song                False
duration_ms         False
explicit            False
year                False
popularity          False
danceability        False
energy              False
key                 False
loudness            False
mode                False
speechiness         False
acousticness        False
instrumentalness    False
liveness            False
valence             False
tempo               False
genre               False
dtype: bool

In [25]:
df_content.sort_values('year')

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
44,Missy Elliott,Hot Boyz,215466,True,1998,49,0.727,0.445,1,-11.241,1,0.2910,0.33900,0.000000,0.1800,0.527,81.125,"hip hop, pop, R&B"
82,Savage Garden,Crash and Burn,281466,False,1999,54,0.581,0.607,4,-8.458,1,0.0280,0.18900,0.000002,0.0882,0.213,102.030,pop
123,Crazy Town,Butterfly,216733,False,1999,71,0.736,0.811,9,-4.170,0,0.0810,0.00132,0.000142,0.1070,0.609,103.502,"rock, metal"
34,Britney Spears,Born to Make You Happy,243533,False,1999,58,0.633,0.922,11,-4.842,0,0.0454,0.11600,0.000465,0.0710,0.686,84.110,pop
35,Montell Jordan,Get It On Tonite,276266,False,1999,59,0.813,0.491,10,-9.923,0,0.0770,0.24100,0.000046,0.0817,0.868,99.008,"hip hop, pop, R&B"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Lil' Kim,Magic Stick,359973,True,2019,47,0.849,0.498,2,-7.872,1,0.2720,0.11600,0.000045,0.2680,0.502,92.980,"hip hop, pop, R&B"
1999,Post Malone,Circles,215280,False,2019,85,0.695,0.762,0,-3.497,1,0.0395,0.19200,0.002440,0.0863,0.553,120.042,hip hop
568,Will Smith,Switch,197666,False,2020,34,0.873,0.900,5,-4.325,0,0.1800,0.01070,0.000502,0.5560,0.478,102.516,"hip hop, pop"
308,t.A.T.u.,All The Things She Said,214440,True,2020,39,0.527,0.834,5,-5.767,0,0.0474,0.04110,0.005990,0.1050,0.381,179.920,pop


In [26]:
df_content_final= df_content

In [27]:
df_collab_final.to_csv('df_collab_cleaned.csv',index=False)
df_content_final.to_csv('df_content_cleaned.csv', index=False)